In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("spotify_millsongdata.csv")

In [3]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [4]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [5]:
df.shape

(57650, 4)

In [6]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [7]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [8]:
df.head(10)

,artist,song,text
0,Everclear,Santa Ana Wind,I feel safe inside the violence \r\nLike I fe...
1,Chris Rea,September Blue,Your head spins round on a Monday \r\nAnd the...
2,Counting Crows,A Little Discipline,This time is better than last time \r\nI feel...
3,Fleetwood Mac,Silver Springs,You could be my silver spring \r\nBlue-green ...
4,Randy Travis,Blue Mesa,Blue Mesa \r\nLand of the Sioux and Crow \r\...
5,George Strait,Marina Del Rey,"We said goodbye in Marina del Rey \r\n""I had ..."
6,Kylie Minogue,Mighty Rivers,I was so close to the edge \r\nI was falling ...
7,Chuck Berry,Every Day I Have The Blues,"Every day, every day I have the blues, \r\nEv..."
8,J Cole,You Got It,"[Refrain: \r\nHey one time, hey one time \r\..."
9,Paul Simon,Train In The Distance,She was beautiful as southern skies the night ...


In [9]:
df['text'][0]

"I feel safe inside the violence  \r\nLike I feel safe inside a roller coaster car  \r\nI feel safe in the hills  \r\nUp above it all sometimes  \r\nIn the middle of the night, I can almost see the stars  \r\n  \r\nLiving in L.A. makes me think of you  \r\nI think about the good times doing all the bad things we used to do  \r\nWalked the streets and I could remember why I feel so bad  \r\nWalked the streets at night, it helps sometimes to remember why  \r\n  \r\nLiving in L.A. feels like home in a brand new way  \r\nI am like everyone I meet,  \r\nI have ghosts in my mind they will never go away  \r\nI walked the streets at night to understand why I feel so bad  \r\nI walked the streets at night questioning why I am the way I am  \r\n  \r\nWe come from the somewhere lost time out in the cold  \r\nWe run for the light, it's the only way we know  \r\nWanna find a better sun, I wanna find a better world  \r\nWanna find a better heart, so I can find me a better girl  \r\nWanna find a plac

In [18]:
df = df.sample(5000)

In [19]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [20]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [32]:
import nltk
from nltk.stem.porter import PorterStemmer
nltk.download('punkt')

stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ACER\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


In [33]:
df['text'] = df['text'].apply(lambda x: tokenization(x))

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [35]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [36]:
similarity[0]

array([1.        , 0.        , 0.04956978, ..., 0.00313137, 0.00222032,
       0.00352831])

In [40]:
df[df['song'] == 'Train In The Distance']

,artist,song,text
9,Paul Simon,Train In The Distance,she wa beauti as southern sky the night he met...


In [43]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [45]:
recommendation('Train In The Distance')

['Already Gone',
 'Thinking Of You',
 "I'm Ready",
 'Ready To Rock',
 'Duck Tales Theme Song',
 "I'm Ready",
 'Ready Or Not',
 'Less Than Zero',
 'See How The Love Goes',
 'American Music',
 'Together',
 'Control',
 "You Can't Run",
 'But You Know I Love You',
 'Gypsy',
 'My Love Life',
 'How Will I Know',
 'Faster',
 'Can Anyone Explain? (No, No, No!)',
 'Hold The Line']

In [46]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))